### Ejercicio 1 – Precio promedio por barrio

Enunciado:

Agrupa por barrio y calcula el precio medio de las viviendas. ¿Qué barrio es el más caro en promedio?

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('Dataset_vivienda_en_Madrid.csv')
df.sample(5)

In [ ]:
df_agrupado=df.groupby("barrio")["precio"].mean().sort_values(ascending=False)
df_agrupado

### Ejercicio 2 – Precio medio por metro cuadrado

Enunciado:

Agrega una nueva columna precio_m2 y calcula el promedio de este valor por barrio. Usa groupby + apply.

In [ ]:
df["precio_m2"] = round(df["precio"] / df["metros_cuadrados"])
df.head(5)


In [ ]:
df.groupby("barrio")["precio_m2"].apply(np.mean).sort_values(ascending=False).round()

### Ejercicio 3 – Clasificación por cuartiles de precio

Enunciado:

Agrupa por barrio, calcula el precio promedio y asigna a cada barrio un cuartil (Q1–Q4) según el valor medio.

In [ ]:
precio_prom = df.groupby("barrio")["precio"].mean()
precio_prom


In [ ]:
cuartiles = pd.qcut(precio_prom, q=4, labels=["Q1", "Q2", "Q3", "Q4"])
cuartiles


In [ ]:
df['barrio']

In [ ]:
df["cuartil_barrio"] = df["barrio"].map(cuartiles)


In [ ]:
df.head(5)

### Ejercicio 4 – Detectar viviendas “anómalas” en su barrio

Enunciado:

Una vivienda es “anómala” si su precio se aleja más de 2 desviaciones estándar del promedio de su barrio. Crea una nueva columna que indique esto.

In [ ]:
def detectar_anomalias(grupo):
    media = grupo["precio"].mean()
    std = grupo["precio"].std()
    return grupo["precio"].apply(lambda x: abs(x - media) < 2 * std)



In [ ]:
df["anomalia"] = df.groupby("barrio").apply(detectar_anomalias).reset_index(drop=True)


In [ ]:
df_anomalas = df[df["anomalia"] == True]
df_anomalas

### Ejercicio 5 – Mapa con folium: precios por color

Enunciado:

Crea un mapa con folium donde cada vivienda tenga un color según su precio:

Verde: <300.000 €

Naranja: 300.000–600.000 €

Rojo: >600.000 €

In [ ]:
import folium

def color_precio(precio):
    if precio < 300000:
        return "green"
    elif precio < 600000:
        return "orange"
    else:
        return "red"

In [ ]:
mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

In [ ]:
df.head()

In [ ]:
for _, fila in df.iterrows():
    folium.CircleMarker(
        location=[fila['latitud'], fila['longitud']],
        radius=5,
        color=color_precio(fila['precio']),
        fill=True,
        fill_opacity=0.6,
        popup=f"Barrio:{fila['barrio']} -Precio: {fila['precio']} € - Habitaciones:{fila['habitaciones']}, - Baños{fila['baños']}"
    ).add_to(mapa)

mapa.save("mapa_colores_precio.html")

### Ejercicio 6 – Mapa de calor de viviendas caras

Enunciado:

Crea un mapa de calor con folium.plugins.HeatMap que indique la concentración de viviendas con precio superior a 600.000 €.

In [ ]:
from folium.plugins import HeatMap

mapa_calor = folium.Map(location=[40.4168, -3.7038], zoom_start=12)#localiacion madrid

In [ ]:
data_heat = df[df["precio"] > 600000][["latitud", "longitud"]].values.tolist()
data_heat

In [ ]:
HeatMap(data_heat, radius=15, blur=10).add_to(mapa_calor)
mapa_calor.save("mapa_calor_viviendas_caras.html")

### Ejercicio 7 – Vivienda más barata por barrio

Enunciado:

Encuentra la vivienda más barata de cada barrio (precio mínimo). Muestra su ubicación en un mapa.

In [ ]:
viviendas_baratas = df.loc[df.groupby("barrio")["precio"].idxmin()]
viviendas_baratas




In [ ]:
mapa_min = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

In [ ]:
for _, row in viviendas_baratas.iterrows():
    folium.Marker(
        location=[row["latitud"], row["longitud"]],
        popup=f"Barrio:{row['barrio']} -Precio: {row['precio']} € - Habitaciones:{row['habitaciones']}, - Baños{row['baños']}",
        icon=folium.Icon(color="blue", icon="home")
    ).add_to(mapa_min)



In [ ]:
mapa_min.save("mapa_viviendas_baratas.html")